
## Crop preparation for   model input file prep

The crop files can be obtained from: You, L., U. Wood-Sichra, S. Fritz, Z. Guo, L. See, and J. Koo. 2014. Spatial Production Allocation Model (SPAM) 2010 V1r1. [October 8, 2019]. Available from http://mapspam.info and on IFPRI’s Dataverse Site.


This code prepares the files based on the downloaded data. It extracts all necessary attributes for the specific location. It also applies some name fixes as needed for the model to run smoothly.The output dataframe is exported as csv and id ready to be used in the irrigation model

## Select the target coordinate system

NOTE:When selecting your coordinate system make sure that you select a system with the unit of meters, this is indicated for all systems on [epsg.io](https://epsg.io/)

In [ ]:
crs = 'EPSG:3395'

## Function required to clip the raster to specific administrative boundaries

In [ ]:
def ClipRasterByMask(raster, mask, crs, output):
    with fiona.open(mask, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    with rasterio.open(raster) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_image[out_image<0] = np.nan
        mask = (out_image!=0)
        out_image = fillnodata(out_image, mask)
        out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "crs": crs})
    
    out_meta.update(compress = 'lzw')
    
    with rasterio.open(output, "w", **out_meta) as dest:
        dest.write(out_image)
    out = rasterio.open(output)
        
    return out

In [ ]:
elecPop = ClipRasterByMask(poprasterio.name, workspace + r"/NTLArea.shp", "EPSG:4326", workspace + r"/rasterBase.tif")

## Select the output folder and the administrative boundaries

In [ ]:
messagebox.showinfo('Crop extraction', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('Crop extraction', 'Select the admin map')
filename_admin = (filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
admin=gpd.read_file(filename_admin)

## Select the crop data folder for processing the files

In [ ]:
raster_path = r"N:\Agrodem\Irrigation_model\Input_data\Supporting_Layers"
raster_files =[]
csvoutpath = r"N:\Agrodem\Irrigation_model\Output_data\agrodem_input\1km_2030_Cassava\SG"

print ("Reading independent variables...")

for i in os.listdir(raster_path):
    if i.endswith('.tif'):
        raster_files.append(i) 
        

# Convert country inputs(csv) to shapefile 

In [ ]:
input = pd.read_csv(input_file)
# Add geometry and convert to spatial dataframe in source CRS
crop_df['geometry'] = list(zip(crop_df['lon'], crop_df['lat']))
crop_df['geometry'] = crop_df['geometry'].apply(Point)
crop_df = gpd.GeoDataFrame(crop_df, geometry='geometry', crs={'init': 'epsg:4326'})

crop_gdf = crop_df.to_crs({'init': 'epsg:32737'})

In [ ]:
:
messagebox.showinfo('OnSSET', 'Select the clusters')
clusters = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

# Processing rasters

In [ ]:

def processing_raster(name, method, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    raster=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))
    
    clusters = zonal_stats(
        clusters,
        raster.name,
        stats=[method],
        prefix=name, geojson_out=True, all_touched=True)
    
    print(datetime.datetime.now())
    return clusters